In [1]:
import os
import cv2
#import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Hi', 'I am', 'From', 'Singapore', 'Paiseh', 'I', 'Cannot', 'Speak', 'But', 'Machine', 'Learning', 'Can', 'Help', 'Translate'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [3]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
loss = []
accuracies = []
precisions = []
recalls = []

kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X):
    
    np.random.shuffle(train_index)
    np.random.shuffle(test_index)
    
    features_train, features_test = X[train_index], X[test_index]
    labels_train, labels_test = y[train_index], y[test_index]
    
    # Construct the required LSTM model.
    model = Sequential() # changed activation to tanh
    model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,1662)))
    model.add(LSTM(128, return_sequences=True, activation='tanh'))
    model.add(LSTM(64, return_sequences=False, activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    # Display the success message.
    print("Model Created Successfully!")
    
    # Create an Instance of Early Stopping Callback
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['categorical_accuracy',Precision(),Recall()]) 
    
    model_training_history = model.fit(X_train, y_train, epochs=300, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, 
                                                     callbacks = [early_stopping_callback])

    # Evaluate the trained model.
    model_evaluation_history = model.evaluate(features_test, labels_test)
    
    eval_loss, eval_acc, eval_prec, eval_rec = model_evaluation_history
    loss.append(eval_loss)
    accuracies.append(eval_acc)
    precisions.append(eval_prec)
    recalls.append(eval_rec)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 128, 128, 16)  448      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 30, 32, 32, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 32, 32, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 30, 32, 32, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 30, 8, 8, 32)     0

Epoch 23/120
112/112 [==============================] - 31s 279ms/step - loss: 0.4808 - accuracy: 0.8705 - precision: 0.9388 - recall: 0.7879 - val_loss: 0.6118 - val_accuracy: 0.8482 - val_precision: 0.9247 - val_recall: 0.7679
Epoch 24/120
112/112 [==============================] - 31s 276ms/step - loss: 0.3649 - accuracy: 0.9174 - precision: 0.9549 - recall: 0.8504 - val_loss: 0.5434 - val_accuracy: 0.8661 - val_precision: 0.8980 - val_recall: 0.7857
Epoch 25/120
112/112 [==============================] - 30s 268ms/step - loss: 0.3620 - accuracy: 0.9018 - precision: 0.9620 - recall: 0.8482 - val_loss: 0.6125 - val_accuracy: 0.8304 - val_precision: 0.9022 - val_recall: 0.7411
Epoch 26/120
112/112 [==============================] - 31s 277ms/step - loss: 0.3304 - accuracy: 0.9152 - precision: 0.9513 - recall: 0.8728 - val_loss: 0.5789 - val_accuracy: 0.8393 - val_precision: 0.8654 - val_recall: 0.8036
Epoch 27/120
112/112 [==============================] - 31s 275ms/step - loss: 0.265

In [ ]:
print("Loss:", loss)
print("Accuracies", accuracies)
print("Precisions", precisions)
print("Recalls", recalls)

In [8]:
print("Mean Loss:", sum(loss)/len(loss))
print("Mean Accuracies", sum(accuracies)/len(accuracies))
print("Mean Precisions", sum(precisions)/len(precisions))
print("Mean Recalls", sum(recalls)/len(recalls))